In [1]:
%load_ext autoreload
%autoreload 2

try:
    from google.colab import drive
    drive.mount('/content/drive')
    import os
    GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'Vgg16'
    GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
    print(os.listdir(GOOGLE_DRIVE_PATH))
    import sys
    sys.path.append(GOOGLE_DRIVE_PATH)
except:
    pass

import time, os, torch, torchvision, random, time, math
from torch import Tensor
import torchvision
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (6, 4)
plt.rcParams['font.size'] = 10
from toolset import *
from toolset.utils import *
from toolset.data import *
from toolset.helper import *
from toolset.solver import *
from convolutional_networks import *
from fully_connected_networks import *

train_data, train_labels, test_data, test_labels = get_mnist_data()

if torch.cuda.is_available:
    print('Good to go!')
else:
    print('Please set GPU via Edit -> Notebook Settings.')

Good to go!


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
train_data = train_data.to(device)
train_labels = train_labels.to(device)
test_data = test_data.to(device)
test_labels = test_labels.to(device)

data_dict = {
    'X_train': train_data,
    'y_train': train_labels,
    'X_val': test_data,
    'y_val': test_labels,
}
print("训练数据集的形态:", train_data.shape)
clear()

cuda
训练数据集的形态: torch.Size([60000, 1, 28, 28])


In [4]:
reset_seed(0)
net = ManualThreeLayerConvNet(
    input_dims = (1, 28, 28),
    num_filters = 32,
    hidden_dim = 128,
    num_classes = 10,
    weight_scale = 1e-3,
    reg = 1e-5,
    device='cuda'
)
solver = Solver(net, data_dict,
                num_epochs=3, batch_size=50,
                optim_config={
                'learning_rate': 0.001,
                },
                update_rule=adam,
                verbose=True, device='cuda',print_every=100)

In [5]:
reset_seed(0)
solver.train(time_limit=600)
# 你的模型1epoch应该能达到95%+准确率

(用时 0.62s; 迭代次数1 / 3600) loss: 2.302594
(Epoch 0 / 3, train 准确率: 15.40%, val准确率: 17.20%
更新:当前准确率17.20, 大于之前的最佳准确率0.00

(用时 1.73s; 迭代次数101 / 3600) loss: 0.233307
(用时 2.75s; 迭代次数201 / 3600) loss: 0.210119
(用时 3.82s; 迭代次数301 / 3600) loss: 0.191380
(用时 4.84s; 迭代次数401 / 3600) loss: 0.125064
(用时 5.81s; 迭代次数501 / 3600) loss: 0.086856
(用时 6.87s; 迭代次数601 / 3600) loss: 0.034448
(用时 7.88s; 迭代次数701 / 3600) loss: 0.066031
(用时 8.91s; 迭代次数801 / 3600) loss: 0.107697
(用时 9.97s; 迭代次数901 / 3600) loss: 0.029145
(用时 10.99s; 迭代次数1001 / 3600) loss: 0.068469
(用时 11.97s; 迭代次数1101 / 3600) loss: 0.103766
(Epoch 1 / 3, train 准确率: 97.70%, val准确率: 97.62%
更新:当前准确率97.62, 大于之前的最佳准确率17.20

(用时 13.06s; 迭代次数1201 / 3600) loss: 0.198591
(用时 14.03s; 迭代次数1301 / 3600) loss: 0.126243
(用时 14.99s; 迭代次数1401 / 3600) loss: 0.013889
(用时 15.96s; 迭代次数1501 / 3600) loss: 0.133625
(用时 16.92s; 迭代次数1601 / 3600) loss: 0.025765
(用时 17.90s; 迭代次数1701 / 3600) loss: 0.063145
(用时 18.87s; 迭代次数1801 / 3600) loss: 0.027185
(用时 19.84s; 迭代次数1901 / 3600

In [ ]:
# 保存权重模型
net.save("mnist_5_27.pth")

In [ ]:
# 加载检测
net_mnist = ManualThreeLayerConvNet()
net_mnist.load("mnist_5_27.pth")

solver_checker = Solver(net, data_dict,optim_config={
                'learning_rate': 0.001,
                },device=device)
acc = solver_checker.check_accuracy(data_dict['X_val'],data_dict['y_val'])
print(acc)